In [1]:
import pandas as pd
import numpy as np
import random
import matplotlib.pyplot as plt
import mlflow
import lightgbm as lgb
import category_encoders as ce
from sklearn.pipeline import make_pipeline
from sklearn.pipeline import Pipeline
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from xgboost import XGBClassifier
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from sklearn.utils import shuffle
from sklearn.neighbors import KNeighborsClassifier

In [2]:
data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')
data.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


## Functions

In [3]:
def get_title(name):
    title=''
    for i in name:
        if i==' ':
            title=''
        elif i=='.':
            return title
        else:
            title=title+i
    return 'Without title'
def Cabin_null_processing(name):
    if (type(name) is int):
        return 0
    else:
        return 1
def Age_null_processing1(Age):
    return 37
def Age_null_processing2(Age):
    return 29
def Age_null_processing3(Age):
    return 27

## Null values processing

In [4]:
#Embarked
data['Embarked']=data['Embarked'].replace(np.nan, 'S')
test_data['Embarked']=test_data['Embarked'].replace(np.nan, 'S')

#Cabin
data['Cabin']=data['Cabin'].replace(np.nan, 0)
data['Cabin']=data['Cabin'].map(Cabin_null_processing)
data['Cabin']=data['Cabin'].replace(np.nan, 0)
data['Cabin']=data['Cabin'].map(Cabin_null_processing)
test_data['Cabin']=test_data['Cabin'].replace(np.nan, 0)
test_data['Cabin']=test_data['Cabin'].map(Cabin_null_processing)
test_data['Cabin']=test_data['Cabin'].replace(np.nan, 0)
test_data['Cabin']=test_data['Cabin'].map(Cabin_null_processing)
#Age
for i in  range(len(data)):
    if (pd.isna(data.loc[i, 'Age'])==True):
        if (data.loc[i,'Pclass']==1):
            data.loc[i,'Age'] = 37
        elif (data.loc[i,'Pclass']==2):
            data.loc[i,'Age'] = 29
        elif (data.loc[i,'Pclass']==3):
            data.loc[i,'Age'] = 27
for i in  range(len(test_data)):
    if (pd.isna(test_data.loc[i, 'Age'])==True):
        if (test_data.loc[i,'Pclass']==1):
            test_data.loc[i,'Age'] = 37
        elif (test_data.loc[i,'Pclass']==2):
            test_data.loc[i,'Age'] = 29
        elif (test_data.loc[i,'Pclass']==3):
            test_data.loc[i,'Age'] = 27            
#Fare
test_data['Fare']=test_data['Fare'].replace(np.nan, 20)

## Data processing

In [5]:
#Sex
data.Sex=data.Sex.replace({'male': 1, 'female' : 0})
test_data.Sex=test_data.Sex.replace({'male': 1, 'female' : 0})

In [6]:
#Ticket
data=data.drop(columns=['Ticket'])
test_data=test_data.drop(columns=['Ticket'])
#data['Ticket']=LabelEncoder().fit_transform(data.Ticket)
#data1['Ticket']=LabelEncoder().fit_transform(data1.Ticket)

In [7]:
#New feature 'Title' & drop 'Name'
data['Title']=data['Name'].map(get_title)
data=data.drop(columns=['Name'])
test_data['Title']=test_data['Name'].map(get_title)
test_data=test_data.drop(columns=['Name'])

In [8]:
#Embarked
data['Embarked']=data['Embarked'].replace({'Q':1, 'C':2, 'S':3})
test_data['Embarked']=test_data['Embarked'].replace({'Q':1, 'C':2, 'S':3})

In [9]:
#PassengerId
#data=data.drop(columns='PassengerId')
#data1=data1.drop(columns='PassengerId')

In [10]:
#Title
data['Title']=LabelEncoder().fit_transform(data.Title)
test_data['Title']=LabelEncoder().fit_transform(test_data.Title)

## Data exploration

In [11]:
#data.groupby('Pclass')[['Survived']].mean()

In [12]:
#data.groupby('Sex')[['Survived']].mean()

In [16]:
#data.groupby('Embarked')[['Survived']].mean()


In [18]:
#data.groupby(['Embarked', 'Pclass', 'Survived']).size().unstack(fill_value=0)

In [ ]:
#data.groupby(['Pclass'])[['Fare']].median()

In [ ]:
#data.groupby(['SibSp'])[['Fare']].median()

In [ ]:
#data.groupby(['SibSp'])[['Survived']].mean()

In [ ]:
#data.groupby(['Parch'])[['Survived']].mean()

In [ ]:
#data.groupby(['Pclass'])[['Age']].mean()

In [ ]:
#data.info()

In [ ]:
#data[pd.isna(data['Cabin'])==False]['Survived'].mean() 

In [ ]:
#data.groupby('Embarked').size()

In [ ]:
#data.groupby('Cabin').size()

In [ ]:
#data.groupby(['Pclass'])[['Age']].median()

In [ ]:
#data.groupby(['Title'])[['Survived']].mean()

In [ ]:
#test_data.info()

## Visualising data

In [ ]:
names = ['male', 'female']
counts = data.Sex.value_counts(['male','female'])
plt.figure(figsize=(2, 5))
plt.ylabel('Survived')
plt.ylim([0, 1])
plt.bar(names, [(data.Sex.value_counts().iloc[1])/data.Sex.size,(data.Sex.value_counts().iloc[0])/data.Sex.size])

In [ ]:
Fare = list(data.Fare.array)
Age = list(data.Age.array)
plt.xlabel('Fare')
plt.ylabel('Age')
print(type(Age))
plt.plot(Fare, Age, 'ro')

In [ ]:
Parch = list(data.Parch.array)
Quantity=[0 for i in range(7)]
#print(Parch)
#print(Quanitity)
for i in np.unique(Parch):
    Quantity[i]=np.sum(Parch==i)
Parch=np.unique(Parch)
plt.xlabel('Parch')
plt.ylabel('Quantity')
plt.bar(Parch, Quantity)

In [ ]:
SibSp = list(data.SibSp.array)
Quantity=[0 for i in range(7)]
for j,i in enumerate(np.unique(SibSp)):
    Quantity[j]=np.sum(SibSp==i)
SibSp=np.unique(SibSp)
print(Quantity)
print(SibSp)
plt.xlabel('SibSp')
plt.ylabel('Quantity')
plt.bar(SibSp, Quantity)

## Split dataset

In [12]:
train_labels=data['Survived']
train_data=data.drop(columns='Survived')
data = shuffle(data)
val_n=int(0.75*len(data))
train_data, val_data = np.split(data,[val_n])
train_labels=train_data['Survived']
val_labels=val_data['Survived']
train_data=train_data.drop(columns=['Survived'])
val_data=val_data.drop(columns=['Survived'])
print(len(train_data))

668


In [51]:
train_data.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Title
0,1,3,1,22.0,1,0,7.2500,0,3,12
1,2,1,0,38.0,1,0,71.2833,0,2,13
2,3,3,0,26.0,0,0,7.9250,0,3,9
3,4,1,0,35.0,1,0,53.1000,0,3,13
4,5,3,1,35.0,0,0,8.0500,0,3,12


In [56]:
val_data.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Cabin,Embarked,Title
84,85,2,0,17.0,0,0,10.5000,0,3,9
869,870,3,1,4.0,1,1,11.1333,0,3,8
94,95,3,1,59.0,0,0,7.2500,0,3,12
829,830,1,0,62.0,0,0,80.0000,0,3,13
780,781,3,0,13.0,0,0,7.2292,0,2,9


## Stratify split

In [14]:
#rand = np.random.RandomState(10)
#shuffle=rand.permutation(len(data))
#train_data, val_data, train_labels, val_labels = train_test_split(data.drop(columns='Survived'), data['Survived'],test_size=0.2, random_state=1, stratify=data['Survived'])
#print(len(train_data))

712


## Random Forest

In [14]:
params = {'bootstrap': True,
 'max_depth': 90,
 'max_features': 'auto',
 'min_samples_leaf': 4,
 'min_samples_split': 2,
 'n_estimators': 500}
model = RandomForestClassifier(**params)
model.fit(train_data, train_labels)
predictions_RF = model.predict(test_data)
print('accuracy: ', accuracy_score(model.predict(val_data), val_labels))


accuracy:  0.8161434977578476


#### Grid Search(RF)

In [14]:
params = {'bootstrap': [True, False],
 'max_depth': [10, 20, 30, 40, 50, 60, 70, 80, 90, 100, None],
 'max_features': ['auto', 'sqrt'],
 'min_samples_leaf': [1, 2, 4],
 'min_samples_split': [2, 5, 10],
 'n_estimators': [1000, 1200, 2000]}#, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000]}

In [15]:
GSmodel=RandomForestClassifier()
GridS = GridSearchCV(GSmodel,params, cv=3,verbose=2,n_jobs=-1)
GridS.fit(train_data, train_labels)
print('accuracy: ', accuracy_score(GridS.predict(val_data), val_labels))

Fitting 3 folds for each of 1188 candidates, totalling 3564 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:   22.0s
[Parallel(n_jobs=-1)]: Done 154 tasks      | elapsed:  1.5min
[Parallel(n_jobs=-1)]: Done 357 tasks      | elapsed:  3.5min
[Parallel(n_jobs=-1)]: Done 640 tasks      | elapsed:  6.2min
[Parallel(n_jobs=-1)]: Done 1005 tasks      | elapsed:  9.6min
[Parallel(n_jobs=-1)]: Done 1450 tasks      | elapsed: 13.9min
[Parallel(n_jobs=-1)]: Done 1977 tasks      | elapsed: 18.6min
[Parallel(n_jobs=-1)]: Done 2584 tasks      | elapsed: 23.6min
[Parallel(n_jobs=-1)]: Done 3273 tasks      | elapsed: 29.2min
[Parallel(n_jobs=-1)]: Done 3564 out of 3564 | elapsed: 31.7min finished


accuracy:  0.8222222222222222


In [16]:
GridS.best_params_

{'bootstrap': True,
 'max_depth': None,
 'max_features': 'auto',
 'min_samples_leaf': 4,
 'min_samples_split': 5,
 'n_estimators': 1200}

#### Search with mlflow

In [57]:
params = {'bootstrap': True,
 'max_depth': None,
 'max_features': 'auto',
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'n_estimators': 100,
}
mlflow.set_experiment('RF')
with mlflow.start_run():
    model=RandomForestClassifier(**params)
    model.fit(train_data, train_labels)
    accuracy = cross_val_score(model, train_data, train_labels, scoring='accuracy', cv = 10)
    mlflow.log_param('params', params)
    mlflow.log_metric('accuracy', accuracy.mean())
    print(accuracy.mean())


0.8189054726368159


#### accuracy on val_data

In [58]:
params ={'bootstrap': True,
 'max_depth': 90,
 'max_features': 'auto',
 'min_samples_leaf': 4,
 'min_samples_split': 2,
 'n_estimators': 500}
model=RandomForestClassifier(**params)
model.fit(train_data, train_labels)
print('accuracy', accuracy_score(model.predict(val_data),val_labels))
print(model.predict(val_data))

accuracy 0.8340807174887892
[1 1 0 1 1 0 0 0 0 1 1 0 1 0 0 0 0 1 0 1 0 0 1 0 0 0 1 0 0 0 0 0 1 0 1 1 0
 0 1 0 1 0 0 1 0 1 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 0 1 0
 1 0 0 0 0 1 1 0 0 0 0 0 0 1 0 0 0 1 1 1 1 0 0 0 0 1 0 0 0 1 1 0 0 0 0 0 0
 1 0 1 1 0 1 0 1 0 0 0 0 0 0 0 1 0 1 0 0 1 0 1 0 0 0 0 0 1 0 0 0 1 1 1 0 0
 0 0 1 0 0 0 0 0 0 1 0 0 0 0 0 0 0 1 1 0 1 0 0 0 0 1 0 0 0 0 0 0 0 0 0 0 1
 1 1 0 0 0 0 0 1 0 0 0 0 0 1 1 1 0 0 0 0 0 0 1 0 0 1 1 0 1 0 0 0 0 1 0 0 1
 0]


## XGB

In [15]:
params={'colsample_bytree': 0.8,
 'max_depth': 5,
 'min_child_weight': 9.0,
 'subsample': 0.9}
model = XGBClassifier(**params)
model.fit(train_data, train_labels)
predictions_XGB = model.predict(test_data)
#print('accuracy: ', accuracy_score(model.predict(test_data), test_labels))

[13:05:53] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.


D:\anaconda\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


#### Grid search(XGB)

In [34]:
#params = {
#        'min_child_weight': [1, 5, 10],
#        'gamma': [0.5, 1, 1.5, 2, 5],
#        'subsample': [0.6, 0.8, 1.0],
#        'colsample_bytree': [0.6, 0.8, 1.0],
#        'max_depth': [3, 4, 5],
#        'n_estimators': [200,400]
#        }
params = {
        'min_child_weight': np.linspace(1,10,10),
        'subsample': np.linspace(0.5,1,6),
        'colsample_bytree': np.linspace(0.5,1,6),
        'max_depth': np.linspace(3,10,7).astype('int32'),
        }

In [35]:
GSmodel=XGBClassifier()
GridS = GridSearchCV(GSmodel,params, cv=3,verbose=2,n_jobs=-1)
GridS.fit(train_data, train_labels)
print('accuracy: ', accuracy_score(GridS.predict(val_data), val_labels))

Fitting 3 folds for each of 2520 candidates, totalling 7560 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  58 tasks      | elapsed:    1.0s
[Parallel(n_jobs=-1)]: Done 300 tasks      | elapsed:    5.1s
[Parallel(n_jobs=-1)]: Done 706 tasks      | elapsed:   15.3s
[Parallel(n_jobs=-1)]: Done 1272 tasks      | elapsed:   28.0s
[Parallel(n_jobs=-1)]: Done 2002 tasks      | elapsed:   43.1s
[Parallel(n_jobs=-1)]: Done 2892 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 3946 tasks      | elapsed:  1.4min
[Parallel(n_jobs=-1)]: Done 5160 tasks      | elapsed:  1.9min
[Parallel(n_jobs=-1)]: Done 6538 tasks      | elapsed:  2.5min


[12:21:11] WARNING: C:/Users/Administrator/workspace/xgboost-win64_release_1.3.0/src/learner.cc:1061: Starting in XGBoost 1.3.0, the default evaluation metric used with the objective 'binary:logistic' was changed from 'error' to 'logloss'. Explicitly set eval_metric if you'd like to restore the old behavior.
accuracy:  0.8268156424581006


[Parallel(n_jobs=-1)]: Done 7560 out of 7560 | elapsed:  2.9min finished
D:\anaconda\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


In [36]:
GridS.best_params_

{'colsample_bytree': 0.8,
 'max_depth': 5,
 'min_child_weight': 9.0,
 'subsample': 0.9}

#### search with mlflow (XGB)

In [62]:
params = {'colsample_bytree': 0.8,
 'max_depth': 5,
 'min_child_weight': 9.0,
 'subsample': 0.9}
mlflow.set_experiment('XGB')
with mlflow.start_run():
    model=XGBClassifier(**params, eval_metric = 'logloss')
    #model.fit(train_data, train_labels, eval_metric = 'logloss')
    accuracy = cross_val_score(model, train_data, train_labels, scoring='accuracy', cv = 10)
    mlflow.log_param('params', params)
    mlflow.log_metric('cv_accuracy', accuracy.mean())
    print('cv_accuracy:', accuracy.mean())
    model=XGBClassifier(**params)
    model.fit(train_data, train_labels,eval_metric = 'logloss')
    print('accuracy on val_data:', accuracy_score(model.predict(val_data),val_labels))
    mlflow.log_metric('accuracy on val_data', accuracy_score(model.predict(val_data),val_labels))


D:\anaconda\lib\site-packages\xgboost\sklearn.py:892: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)


cv_accuracy: 0.8024197195838987
accuracy on val_data: 0.8116591928251121


## Catboost

In [16]:
params ={'depth': 10, 'iterations': 50, 'learning_rate': 0.25   }
model = CatBoostClassifier(**params)
model.fit(train_data, train_labels)
predictions_Catboost = model.predict(test_data)
print("Your submission was successfully saved!")
print('accuracy: ', accuracy_score(model.predict(val_data), val_labels))

0:	learn: 0.6193579	total: 70.6ms	remaining: 3.46s
1:	learn: 0.5716354	total: 71ms	remaining: 1.7s
2:	learn: 0.5322972	total: 84ms	remaining: 1.31s
3:	learn: 0.4956877	total: 84.5ms	remaining: 972ms
4:	learn: 0.4703895	total: 97.2ms	remaining: 875ms
5:	learn: 0.4531190	total: 110ms	remaining: 805ms
6:	learn: 0.4372028	total: 123ms	remaining: 753ms
7:	learn: 0.4260892	total: 135ms	remaining: 710ms
8:	learn: 0.4158603	total: 147ms	remaining: 671ms
9:	learn: 0.4070430	total: 160ms	remaining: 639ms
10:	learn: 0.3984556	total: 166ms	remaining: 587ms
11:	learn: 0.3895414	total: 171ms	remaining: 543ms
12:	learn: 0.3822631	total: 184ms	remaining: 522ms
13:	learn: 0.3767112	total: 184ms	remaining: 473ms
14:	learn: 0.3710761	total: 196ms	remaining: 458ms
15:	learn: 0.3660758	total: 208ms	remaining: 443ms
16:	learn: 0.3611474	total: 211ms	remaining: 410ms
17:	learn: 0.3566061	total: 224ms	remaining: 398ms
18:	learn: 0.3501018	total: 236ms	remaining: 384ms
19:	learn: 0.3455477	total: 248ms	remaini

#### grid search (Catbo0st)

In [86]:
params = {'depth': [4,5,6,7,8,9,10],
                  'learning_rate' : [0.05,0.1,0.2,0.25],
                  'iterations'    : [10,20,30,40,50,60,70,80,90,100]
                 }

In [84]:
GSmodel = CatBoostClassifier(**params)
GridS = GridSearchCV(GSmodel, params, cv=3,verbose=2,n_jobs=-1)
GridS.fit(train_data, train_labels)

Fitting 3 folds for each of 280 candidates, totalling 840 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  33 tasks      | elapsed:    3.5s
[Parallel(n_jobs=-1)]: Done 164 tasks      | elapsed:    9.6s
[Parallel(n_jobs=-1)]: Done 521 tasks      | elapsed:   39.4s
[Parallel(n_jobs=-1)]: Done 804 tasks      | elapsed:  2.0min
[Parallel(n_jobs=-1)]: Done 840 out of 840 | elapsed:  2.5min finished


0:	learn: 0.6325930	total: 62.3ms	remaining: 3.05s
1:	learn: 0.5788265	total: 62.7ms	remaining: 1.5s
2:	learn: 0.5348055	total: 76.7ms	remaining: 1.2s
3:	learn: 0.4987933	total: 77.6ms	remaining: 893ms
4:	learn: 0.4743317	total: 92.3ms	remaining: 830ms
5:	learn: 0.4541888	total: 107ms	remaining: 786ms
6:	learn: 0.4410172	total: 122ms	remaining: 748ms
7:	learn: 0.4288097	total: 136ms	remaining: 713ms
8:	learn: 0.4181826	total: 149ms	remaining: 677ms
9:	learn: 0.4067285	total: 161ms	remaining: 645ms
10:	learn: 0.4003364	total: 162ms	remaining: 575ms
11:	learn: 0.3925937	total: 178ms	remaining: 565ms
12:	learn: 0.3857692	total: 196ms	remaining: 559ms
13:	learn: 0.3814745	total: 197ms	remaining: 506ms
14:	learn: 0.3756712	total: 210ms	remaining: 491ms
15:	learn: 0.3696367	total: 224ms	remaining: 475ms
16:	learn: 0.3632250	total: 237ms	remaining: 461ms
17:	learn: 0.3568226	total: 252ms	remaining: 447ms
18:	learn: 0.3508396	total: 266ms	remaining: 435ms
19:	learn: 0.3462595	total: 280ms	rema

GridSearchCV(cv=3,
             estimator=<catboost.core.CatBoostClassifier object at 0x0000026DE0C64640>,
             n_jobs=-1,
             param_grid={'depth': [4, 5, 6, 7, 8, 9, 10],
                         'iterations': [10, 20, 30, 40, 50, 60, 70, 80, 90,
                                        100],
                         'learning_rate': [0.05, 0.1, 0.2, 0.25]},
             verbose=2)

In [85]:
GridS.best_params_

{'depth': 10, 'iterations': 50, 'learning_rate': 0.25}

## LightGBM

In [17]:
params = {'max_depth': 7, 'min_data_in_leaf': 70, 'num_leaves': 75}
model = LGBMClassifier(**params)
model.fit(train_data, train_labels)
predictions_LGBM = model.predict(test_data)
print('accuracy: ', accuracy_score(model.predict(val_data), val_labels))


[LightGBM] [Warning] min_data_in_leaf is set=70, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=70
accuracy:  0.7982062780269058


In [ ]:
GSmodel = LGBMClassifier()
GridS = GridSearchCV(GSmodel, params, cv=3,verbose=2,n_jobs=-1)
GridS.fit(train_data, train_labels)

#### grid search (LGBM)

In [49]:
params = {
    'num_leaves': [1, 3, 7, 20, 31, 40, 60, 80, 127, 170, 210],
    'max_depth': [1, 3, 5, 7, 9, 15, 20, 25, 40],
    'min_data_in_leaf': [100, 300, 400, 700, 1000, 1300, 1500],
    }

In [50]:
GSmodel = LGBMClassifier()
GridS = GridSearchCV(GSmodel, params, cv=3,verbose=2,n_jobs=-1)
GridS.fit(train_data, train_labels)

Fitting 3 folds for each of 693 candidates, totalling 2079 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done 136 tasks      | elapsed:    0.5s
[Parallel(n_jobs=-1)]: Done 2027 tasks      | elapsed:    6.0s
[Parallel(n_jobs=-1)]: Done 2079 out of 2079 | elapsed:    6.2s finished


GridSearchCV(cv=3, estimator=LGBMClassifier(), n_jobs=-1,
             param_grid={'max_depth': [1, 3, 5, 7, 9, 15, 20, 25, 40],
                         'min_data_in_leaf': [100, 300, 400, 700, 1000, 1300,
                                              1500],
                         'num_leaves': [1, 3, 7, 20, 31, 40, 60, 80, 127, 170,
                                        210]},
             verbose=2)

In [51]:
GridS.best_params_

{'max_depth': 3, 'min_data_in_leaf': 100, 'num_leaves': 7}

#### search with mlflow (LGBM)

In [75]:
params = {'max_depth': 7, 'min_data_in_leaf': 70, 'num_leaves': 75}
mlflow.set_experiment('LGBM')
with mlflow.start_run():
    model=LGBMClassifier(**params)
    #model.fit(train_data, train_labels, eval_metric = 'logloss')
    accuracy = cross_val_score(model, train_data, train_labels, scoring='accuracy', cv = 10)
    mlflow.log_param('params', params)
    mlflow.log_metric('cv_accuracy', accuracy.mean())
    print('cv_accuracy:', accuracy.mean())
    model=LGBMClassifier(**params)
    model.fit(train_data, train_labels)
    print('accuracy on val_data:', accuracy_score(model.predict(val_data),val_labels))
    mlflow.log_metric('accuracy on val_data', accuracy_score(model.predict(val_data),val_labels))

[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] min_data_in_leaf is set=50, min_child_samples=20 will be ignored. Current value: min_data_in_leaf=50
[LightGBM] [Warning] min

## SVM

In [18]:
params = {'C': 5, 'coef0': 0.001, 'degree': 3, 'gamma': 0.0006, 'kernel': 'rbf'}
model = SVC(**params)
clf = make_pipeline(StandardScaler(), model)
clf.fit(train_data, train_labels)
predictions_SVM = clf.predict(test_data)
print('accuracy: ', accuracy_score(clf.predict(val_data), val_labels))

accuracy:  0.8116591928251121


#### gridsearch (SVM)

In [14]:
params = {
    'kernel' : ('linear', 'poly', 'rbf', 'sigmoid'),
    'C' : np.arange(0.1,1,0.4),
    'degree': np.arange(3,10,3),
    'coef0': np.arange(0.001,5,2),
    'gamma': ('auto', 'scale')
}

In [ ]:
GSmodel=SVC()
GridS = GridSearchCV(GSmodel,params, cv=5,verbose=3, n_jobs=-1)
GridS.fit(train_data, train_labels)

Fitting 5 folds for each of 216 candidates, totalling 1080 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.


#### search with mlflow (SVM)

In [33]:
params ={'C': 1, 'coef0': 0.001, 'degree': 3, 'gamma': 'scale', 'kernel': 'rbf'}
mlflow.set_experiment('LGBM')
with mlflow.start_run():
    model=SVC(**params)
    #model.fit(train_data, train_labels, eval_metric = 'logloss')
    clf = make_pipeline(StandardScaler(), model)
    accuracy = cross_val_score(clf, train_data, train_labels, scoring='accuracy', cv = 10)
    mlflow.log_param('params', params)
    mlflow.log_metric('cv_accuracy', accuracy.mean())
    print('cv_accuracy:', accuracy.mean())
    model=SVC(**params)
    clf = make_pipeline(StandardScaler(), model)
    clf.fit(train_data, train_labels)
    print('accuracy on val_data:', accuracy_score(clf.predict(val_data),val_labels))
    mlflow.log_metric('accuracy on val_data', accuracy_score(clf.predict(val_data),val_labels))

cv_accuracy: 0.822003618272275
accuracy on val_data: 0.8385650224215246


## Logistic Regression

In [19]:
params={'C': 10.0, 'penalty': 'l2'}
model = LogisticRegression(**params)
model.fit(train_data, train_labels)
predictions_LogisticRegression = model.predict(test_data)
print("Your submission was successfully saved!")
print('accuracy: ', accuracy_score(model.predict(val_data), val_labels))

Your submission was successfully saved!
accuracy:  0.7892376681614349


D:\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


#### gridsearch(Logistic Regression)

In [94]:
params={"C":np.logspace(-3,3,7), "penalty":["l1","l2"]}

In [95]:
GSmodel=LogisticRegression()
GridS = GridSearchCV(GSmodel,params, cv=5,verbose=3, n_jobs=-1)
GridS.fit(train_data, train_labels)

Fitting 5 folds for each of 14 candidates, totalling 70 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    1.7s
[Parallel(n_jobs=-1)]: Done  63 out of  70 | elapsed:    1.9s remaining:    0.1s
[Parallel(n_jobs=-1)]: Done  70 out of  70 | elapsed:    2.0s finished
D:\anaconda\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(cv=5, estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'C': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02, 1.e+03]),
                         'penalty': ['l1', 'l2']},
             verbose=3)

In [96]:
GridS.best_params_

{'C': 10.0, 'penalty': 'l2'}

## kNN

In [28]:
params={'n_neighbors': 13}
model = KNeighborsClassifier(**params)
clf = make_pipeline(StandardScaler(), model)
clf.fit(train_data, train_labels)
predictions_kNN = clf.predict(test_data)
print("Your submission was successfully saved!")
print('accuracy: ', accuracy_score(clf.predict(val_data), val_labels))

Your submission was successfully saved!
accuracy:  0.8385650224215246


#### gridsearch (kNN)

In [129]:
#print(np.linspace(1,32,32))
params = {'n_neighbors': list(np.linspace(1,32,32).astype('int32'))}


In [133]:
GSmodel=KNeighborsClassifier()
GridS = GridSearchCV(GSmodel,params, cv=5,verbose=3, n_jobs=-1)
GridS.fit(train_data, train_labels)

Fitting 5 folds for each of 32 candidates, totalling 160 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  24 tasks      | elapsed:    1.5s
[Parallel(n_jobs=-1)]: Done 160 out of 160 | elapsed:    2.0s finished


GridSearchCV(cv=5, estimator=KNeighborsClassifier(), n_jobs=-1,
             param_grid={'n_neighbors': [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12,
                                         13, 14, 15, 16, 17, 18, 19, 20, 21, 22,
                                         23, 24, 25, 26, 27, 28, 29, 30, ...]},
             verbose=3)

In [134]:
GridS.best_params_

{'n_neighbors': 26}

## Final predictions

In [33]:
prediction = [0 for i in range(len(test_data))]
print(len(prediction))
for i in range(len(test_data)):
    sum_predictions=predictions_XGB[i]+predictions_SVM[i]+predictions_kNN[i]+predictions_LogisticRegression[i]+predictions_Catboost[i]+predictions_LGBM[i]
    if sum_predictions>3:
        prediction[i]=1
    else:
        prediction[i]=0

print(prediction)
output = pd.DataFrame({'PassengerId': test_data.PassengerId, 'Survived': predictions_LGBM})
output.to_csv('my_submission.csv', index=False)

418
[0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 1, 0, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 1, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0,